# MLP Classification with CR Dataset
<hr>

We will build a text classification model using MLP model on the Customer Reviews Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
from nltk.corpus import stopwords, twitter_samples
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold
from nltk.stem import PorterStemmer
from string import punctuation
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
import time

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')


Bad key text.latex.unicode in file C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get an updated matplotlibrc file from
h

In [2]:
tf.config.experimental.list_physical_devices('GPU')

[]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../0_data/TREC/TREC.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(5952, 3)


sentence  label  split
0     how did serfdom develop in and then leave russ...      0  train
1      what films featured the character popeye doyle ?      1  train
2     how can i find a list of celebrities ' real na...      0  train
3     what fowl grabs the spotlight after the chines...      1  train
4                       what is the full form of .com ?      2  train
...                                                 ...    ...    ...
5947             who was the 22nd president of the us ?      3   test
5948             what is the money they use in zambia ?      1   test
5949                          how many feet in a mile ?      5   test
5950                what is the birthstone of october ?      1   test
5951                                   what is e coli ?      0   test

[5952 rows x 3 columns]

In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5952 entries, 0 to 5951
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  5952 non-null   object
 1   label     5952 non-null   int32 
 2   split     5952 non-null   object
dtypes: int32(1), object(2)
memory usage: 116.4+ KB


In [5]:
corpus.groupby( by=['split','label']).count()

sentence
split label          
test  0           138
      1            94
      2             9
      3            65
      4            81
      5           113
train 0          1162
      1          1250
      2            86
      3          1223
      4           835
      5           896

In [6]:
corpus.groupby(by='split').count()

sentence  label
split                 
test        500    500
train      5452   5452

In [11]:
# Separate the sentences and the labels
# Separate the sentences and the labels for training and testing
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
print(len(train_x))
print(len(train_y))

test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)
print(len(test_x))
print(len(test_y))

5452
5452
500
500


## Raw Number of Vocabulary

In [9]:
# Build the raw vocobulary for first inspection
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_raw = tokenizer.word_index
print('\nThe vocabulary size: {}\n'.format(len(vocab_raw)))
print(vocab_raw)


The vocabulary size: 8759

{'the': 1, 'what': 2, 'is': 3, 'of': 4, 'in': 5, 'a': 6, 'how': 7, "'s": 8, 'was': 9, 'who': 10, 'to': 11, 'are': 12, 'for': 13, 'and': 14, 'did': 15, 'does': 16, "''": 17, 'do': 18, 'name': 19, 'on': 20, 'many': 21, 'where': 22, 'first': 23, 'when': 24, 'i': 25, 'you': 26, 'can': 27, 'from': 28, 'world': 29, 's': 30, 'u': 31, 'which': 32, 'that': 33, 'most': 34, 'by': 35, 'an': 36, 'country': 37, 'as': 38, 'city': 39, 'with': 40, 'have': 41, 'has': 42, 'why': 43, 'it': 44, 'there': 45, 'year': 46, 'state': 47, 'called': 48, 'be': 49, 'president': 50, 'people': 51, 'at': 52, 'get': 53, 'were': 54, 'find': 55, 'his': 56, 'american': 57, 'mean': 58, 'two': 59, 'largest': 60, 'war': 61, 'made': 62, 'new': 63, 'much': 64, 'fear': 65, 'long': 66, 'between': 67, "'": 68, 'its': 69, 'used': 70, 'word': 71, 'known': 72, 'origin': 73, 'day': 74, 'company': 75, 'kind': 76, 'movie': 77, 'about': 78, 'tv': 79, 'one': 80, 'film': 81, 'all': 82, 'famous': 83, 'stand': 84,

<!--## Split Dataset-->

# Data Preprocessing
<hr>

## Define `clean_doc` function

In [12]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stemmer = PorterStemmer()
    
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # remove remaining tokens that are not alphabetic
#     tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    tokens = [w for w in tokens if not w in stopwords]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) >= 1]
    # Stem the token
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens

## Develop Vocabulary

A part of preparing text for text classification involves defining and tailoring the vocabulary of words supported by the model. **We can do this by loading all of the documents in the dataset and building a set of words.**

The larger the vocabulary, the more sparse the representation of each word or document. So, we may decide to support all of these words, or perhaps discard some. The final chosen vocabulary can then be saved to a file for later use, such as filtering words in new documents in the future.

We can use `Counter` class and create an instance called `vocab` as follows:

In [13]:
from collections import Counter

vocab = Counter()

def add_doc_to_vocab(docs, vocab):
    '''
    input:
        docs: a list of sentences (docs)
        vocab: a vocabulary dictionary
    output:
        return an updated vocabulary
    '''
    for doc in docs:
        tokens = clean_doc(doc)
        vocab.update(tokens)
    return vocab

In [14]:
# Example
add_doc_to_vocab(train_x, vocab)
print(len(vocab))
vocab

6840


Counter({'serfdom': 1,
         'develop': 12,
         'leav': 8,
         'russia': 5,
         'film': 57,
         'featur': 29,
         'charact': 42,
         'popey': 4,
         'doyl': 1,
         'find': 87,
         'list': 12,
         'celebr': 14,
         'real': 15,
         'name': 392,
         'fowl': 1,
         'grab': 2,
         'spotlight': 1,
         'chines': 10,
         'year': 114,
         'monkey': 3,
         'full': 10,
         'form': 26,
         'com': 1,
         'contempt': 3,
         'scoundrel': 3,
         'stole': 5,
         'cork': 4,
         'lunch': 3,
         'team': 37,
         'basebal': 33,
         'st': 14,
         'loui': 5,
         'brown': 10,
         'becom': 32,
         'oldest': 12,
         'profess': 5,
         'liver': 2,
         'enzym': 1,
         'scar': 2,
         'face': 15,
         'bounti': 4,
         'hunter': 4,
         'old': 39,
         'west': 20,
         'ozzi': 2,
         'osbourn': 1,
     

In [15]:
vocab.items()

dict_items([('serfdom', 1), ('develop', 12), ('leav', 8), ('russia', 5), ('film', 57), ('featur', 29), ('charact', 42), ('popey', 4), ('doyl', 1), ('find', 87), ('list', 12), ('celebr', 14), ('real', 15), ('name', 392), ('fowl', 1), ('grab', 2), ('spotlight', 1), ('chines', 10), ('year', 114), ('monkey', 3), ('full', 10), ('form', 26), ('com', 1), ('contempt', 3), ('scoundrel', 3), ('stole', 5), ('cork', 4), ('lunch', 3), ('team', 37), ('basebal', 33), ('st', 14), ('loui', 5), ('brown', 10), ('becom', 32), ('oldest', 12), ('profess', 5), ('liver', 2), ('enzym', 1), ('scar', 2), ('face', 15), ('bounti', 4), ('hunter', 4), ('old', 39), ('west', 20), ('ozzi', 2), ('osbourn', 1), ('born', 31), ('heavier', 2), ('object', 6), ('travel', 20), ('downhil', 1), ('faster', 1), ('pride', 2), ('yanke', 9), ('kill', 28), ('gandhi', 4), ('consid', 10), ('costliest', 1), ('disast', 1), ('insur', 5), ('industri', 4), ('ever', 13), ('sprawl', 1), ('us', 166), ('state', 119), ('boast', 16), ('airport', 1

In [16]:
#########################
# Define the vocabulary #
#########################

from collections import Counter
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stemmer = PorterStemmer()
    
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # filter out stop words
    tokens = [w for w in tokens if not w in stopwords]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) >= 1]
    # Stem the token
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens

def add_doc_to_vocab(docs, vocab):
    '''
    input:
        docs: a list of sentences (docs)
        vocab: a vocabulary dictionary
    output:
        return an updated vocabulary
    '''
    for doc in docs:
        tokens = clean_doc(doc)
        vocab.update(tokens)
    return vocab

# Separate the sentences and the labels for training and testing
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
print(len(train_x))
print(len(train_y))

test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)
print(len(test_x))
print(len(test_y))

# Instantiate a vocab object
vocab = Counter()

vocab = add_doc_to_vocab(train_x, vocab)
print(len(train_x), len(test_x))
print(len(vocab))

5452
5452
500
500
5452 500
6840


<dir>

# Bag-of-Words Representation
<hr>

Once we define our vocab obtained from the training data, we need to **convert each review into a representation that we can feed to a Multilayer Perceptron Model.**

As a reminder, here are the summary what we will do:
- extract features from the text so the text input can be used with ML algorithms like neural networks
- we do by converting the text into a vector representation. The larger the vocab, the longer the representation.
- we will score the words in a document inside the vector. These scores are placed in the corresponding location in the vector representation.

In [17]:
def doc_to_line(doc):
    tokens = clean_doc(doc)
    # filter by vocab
    tokens = [token for token in tokens if token in vocab]
    line = ' '.join([token for token in tokens])
    return line

In [18]:
def clean_docs(docs):
    lines = []
    for doc in docs:
        line = doc_to_line(doc)
        lines.append(line)
    return lines

In [19]:
print(train_x[:5])
clean_sentences = clean_docs(train_x[:5])
print()
print( clean_sentences)

['how did serfdom develop in and then leave russia ?', 'what films featured the character popeye doyle ?', "how can i find a list of celebrities ' real names ?", 'what fowl grabs the spotlight after the chinese year of the monkey ?', 'what is the full form of .com ?']

['serfdom develop leav russia', 'film featur charact popey doyl', 'find list celebr real name', 'fowl grab spotlight chines year monkey', 'full form com']


## Bag-of-Words Vectors

We will use the **Keras API** to **convert sentences to encoded document vectors**. Although the `Tokenizer` class from TF Keras provides cleaning and vocab definition, it's better we do this ourselves so that we know exactly we are doing.

In [20]:
def create_tokenizer(sentence):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

This process determines a consistent way to **convert the vocabulary to a fixed-length vector**, which is the total number of words in the vocabulary `vocab`. 

Next, documents can then be encoded using the Tokenizer by calling `texts_to_matrix()`. 

The function takes both a list of documents to encode and an encoding mode, which is the method used to score words in the document. Here we specify **freq** to score words based on their frequency in the document. 

This can be used to encode the loaded training and test data, for example:

`Xtrain = tokenizer.texts_to_matrix(train_docs, mode='freq')`

`Xtest = tokenizer.texts_to_matrix(test_docs, mode='freq')`

In [8]:
#########################
# Define the vocabulary #
#########################

from scipy.io import savemat
from collections import Counter
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stemmer = PorterStemmer()
    
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # filter out stop words
    tokens = [w for w in tokens if not w in stopwords]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) >= 1]
    # Stem the token
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens

def add_doc_to_vocab(docs, vocab):
    '''
    input:
        docs: a list of sentences (docs)
        vocab: a vocabulary dictionary
    output:
        return an updated vocabulary
    '''
    for doc in docs:
        tokens = clean_doc(doc)
        vocab.update(tokens)
    return vocab
        
def doc_to_line(doc, vocab):
    tokens = clean_doc(doc)
    # filter by vocab
    tokens = [token for token in tokens if token in vocab]
    line = ' '.join(tokens)
    return line

def clean_docs(docs, vocab):
    lines = []
    for doc in docs:
        line = doc_to_line(doc, vocab)
        lines.append(line)
    return lines

def create_tokenizer(sentences):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentences)
    return tokenizer

# Separate the sentences and the labels for training and testing
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
print('train_x size: ', len(train_x))
print('train_y size: ', len(train_y))

test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)
print('test_x size: ', len(test_x))
print('test_y size: ', len(test_y))

# Run Experiment of 4 different modes
modes = ['binary', 'count', 'tfidf', 'freq']

for mode in modes:
    print('mode: ', mode)
    
    # Instantiate a vocab object
    vocab = Counter()

    # Define a vocabulary for each fold
    vocab = add_doc_to_vocab(train_x, vocab)
    print('The number of vocab: ', len(vocab))

    # Clean the sentences
    train_x = clean_docs(train_x, vocab)
    test_x = clean_docs(test_x, vocab)

    # Define the tokenizer
    tokenizer = create_tokenizer(train_x)

    # encode data using freq mode
    Xtrain = tokenizer.texts_to_matrix(train_x, mode)
    Xtest = tokenizer.texts_to_matrix(test_x, mode)


    data = {}

    data["Xtrain"] = Xtrain
    data["ytrain"] = train_y
    data["Xtest"] = Xtest
    data["ytest"] = test_y
    
    savemat(mode+"_TREC.mat", data)
    

train_x size:  5452
train_y size:  5452
test_x size:  500
test_y size:  500
mode:  binary
The number of vocab:  6840
mode:  count
The number of vocab:  6799
mode:  tfidf
The number of vocab:  6795
mode:  freq
The number of vocab:  6795


## Save into Matlab format

In [11]:
from scipy.io import savemat
savemat("bow_TREC.mat", data)

In [13]:
data.get('Xtrain').shape

(5452, 6841)

# Training and Testing the Model

## MLP Model

Now, we will build Multilayer Perceptron (MLP) models to classify encoded documents as either positive or negative.

As you might have expected, the models are simply feedforward network with fully connected layers called `Dense` in the `Keras` library.

Now, we will define our MLP neural network with very little trial and error so cannot be considered tuned for this problem. The configuration is as follows:
- First hidden layer with 100 neurons and Relu activation function
- Second hidden layer with 50 neurons and Relu activation function
- Output layer with softmax activation function
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: sparse categorical cross-entropy (suited for multiclass classification problem)

In [24]:
def train_mlp(train_x, train_y, batch_size = 50, epochs = 10, verbose =2):
    
    n_words = train_x.shape[1]
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=100, activation='relu', input_shape=(n_words,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=50, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=6, activation='softmax')
    ])
    
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(train_x, train_y, batch_size, epochs, verbose)
    return model

## Train and Test the Model

In [12]:
#########################
# Define the vocabulary #
#########################

from collections import Counter
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stemmer = PorterStemmer()
    
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # filter out stop words
    tokens = [w for w in tokens if not w in stopwords]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) >= 1]
    # Stem the token
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens

def add_doc_to_vocab(docs, vocab):
    '''
    input:
        docs: a list of sentences (docs)
        vocab: a vocabulary dictionary
    output:
        return an updated vocabulary
    '''
    for doc in docs:
        tokens = clean_doc(doc)
        vocab.update(tokens)
    return vocab
        
def doc_to_line(doc, vocab):
    tokens = clean_doc(doc)
    # filter by vocab
    tokens = [token for token in tokens if token in vocab]
    line = ' '.join(tokens)
    return line

def clean_docs(docs, vocab):
    lines = []
    for doc in docs:
        line = doc_to_line(doc, vocab)
        lines.append(line)
    return lines

def create_tokenizer(sentences):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentences)
    return tokenizer

def train_mlp(train_x, train_y, batch_size = 50, epochs = 10, verbose =2):
    
    n_words = train_x.shape[1]
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=100, activation='relu', input_shape=(n_words,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=50, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=6, activation='softmax')
    ])
    
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(train_x, train_y, batch_size, epochs, verbose)
    return model

# Separate the sentences and the labels for training and testing
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
print('train_x size: ', len(train_x))
print('train_y size: ', len(train_y))

test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)
print('test_x size: ', len(test_x))
print('test_y size: ', len(test_y))

# Instantiate a vocab object
vocab = Counter()
    
# Define a vocabulary for each fold
vocab = add_doc_to_vocab(train_x, vocab)
print('The number of vocab: ', len(vocab))
    
# Clean the sentences
train_x = clean_docs(train_x, vocab)
test_x = clean_docs(test_x, vocab)
    
# Define the tokenizer
tokenizer = create_tokenizer(train_x)
    
# encode data using freq mode
Xtrain = tokenizer.texts_to_matrix(train_x, mode='freq')
Xtest = tokenizer.texts_to_matrix(test_x, mode='freq')

# train the model
model = train_mlp(Xtrain, train_y, epochs = 8)

# evaluate the model
loss, acc = model.evaluate(Xtest, test_y, verbose=0)
print('Test Accuracy: {}'.format(acc*100))

train_x size:  5452
train_y size:  5452
test_x size:  500
test_y size:  500
The number of vocab:  6840
Epoch 1/8
110/110 - 2s - loss: 1.7140 - accuracy: 0.2416
Epoch 2/8
110/110 - 1s - loss: 1.5062 - accuracy: 0.3835
Epoch 3/8
110/110 - 1s - loss: 1.1584 - accuracy: 0.6042
Epoch 4/8
110/110 - 1s - loss: 0.8392 - accuracy: 0.7381
Epoch 5/8
110/110 - 1s - loss: 0.6442 - accuracy: 0.8014
Epoch 6/8
110/110 - 1s - loss: 0.5049 - accuracy: 0.8534
Epoch 7/8
110/110 - 1s - loss: 0.4000 - accuracy: 0.8841
Epoch 8/8
110/110 - 1s - loss: 0.3238 - accuracy: 0.9123
Test Accuracy: 72.00000286102295


In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               684200    
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 306       
Total params: 689,556
Trainable params: 689,556
Non-trainable params: 0
_________________________________________________________________


## Comparing the Word Scoring Methods

When we use `text_to_matrix()` function, we are given 4 different methods for scoring words:
- `binary`: words are marked as 1 (present) or 0 (absent)
- `count`: words are counted based on their occurrence (integer)
- `tfidf`: words are scored  based on their frequency of occurrence in their own document, but also are being penalized if they are common across  all documents
- `freq`: wrods are scored based on their frequency of occurrence in their own document

In [27]:
# prepare bag-of-words encoding of docs
def prepare_data(train_docs, test_docs, mode):
    # create the tokenizer
    tokenizer = Tokenizer()
    # fit the tokenizer on the documents
    tokenizer.fit_on_texts(train_docs)
    # encode training data set
    Xtrain = tokenizer.texts_to_matrix(train_docs, mode=mode)
    # encode test data set
    Xtest = tokenizer.texts_to_matrix(test_docs, mode=mode)
    return Xtrain, Xtest

In [26]:
#########################
# Define the vocabulary #
#########################

from collections import Counter
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stemmer = PorterStemmer()
    
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # filter out stop words
    tokens = [w for w in tokens if not w in stopwords]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) >= 1]
    # Stem the token
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens

def add_doc_to_vocab(docs, vocab):
    '''
    input:
        docs: a list of sentences (docs)
        vocab: a vocabulary dictionary
    output:
        return an updated vocabulary
    '''
    for doc in docs:
        tokens = clean_doc(doc)
        vocab.update(tokens)
    return vocab
        
def doc_to_line(doc, vocab):
    tokens = clean_doc(doc)
    # filter by vocab
    tokens = [token for token in tokens if token in vocab]
    line = ' '.join(tokens)
    return line

def clean_docs(docs, vocab):
    lines = []
    for doc in docs:
        line = doc_to_line(doc, vocab)
        lines.append(line)
    return lines

# prepare bag-of-words encoding of docs
def prepare_data(train_docs, test_docs, mode):
    # create the tokenizer
    tokenizer = Tokenizer()
    # fit the tokenizer on the documents
    tokenizer.fit_on_texts(train_docs)
    # encode training data set
    Xtrain = tokenizer.texts_to_matrix(train_docs, mode=mode)
    # encode test data set
    Xtest = tokenizer.texts_to_matrix(test_docs, mode=mode)
    return Xtrain, Xtest

def train_mlp(train_x, train_y, batch_size = 50, epochs = 10, verbose =2):
    
    n_words = train_x.shape[1]
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=100, activation='relu', input_shape=(n_words,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=50, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=6, activation='softmax')
    ])
    
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(train_x, train_y, batch_size, epochs, verbose)
    return model

# Separate the sentences and the labels for training and testing
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
print('train_x size: ', len(train_x))
print('train_y size: ', len(train_y))

test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)
print('test_x size: ', len(test_x))
print('test_y size: ', len(test_y))

# Run Experiment of 4 different modes
modes = ['binary', 'count', 'tfidf', 'freq']
results = pd.DataFrame()

for mode in modes:
    print('mode: ', mode)
    
    # Instantiate a vocab object
    vocab = Counter()

    # Define a vocabulary for each fold
    vocab = add_doc_to_vocab(train_x, vocab)

    # Clean the sentences
    train_x = clean_docs(train_x, vocab)
    test_x = clean_docs(test_x, vocab)

    # encode data using freq mode
    Xtrain, Xtest = prepare_data(train_x, test_x, mode)

    # train the model
    model = train_mlp(Xtrain, train_y, verbose=0, epochs = 8)

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))
    results[mode] = [acc*100]
    
print()
print(results)

train_x size:  5452
train_y size:  5452
test_x size:  500
test_y size:  500
mode:  binary
Test Accuracy: 73.19999933242798
mode:  count
Test Accuracy: 73.19999933242798
mode:  tfidf
Test Accuracy: 71.20000123977661
mode:  freq
Test Accuracy: 71.39999866485596

      binary      count      tfidf       freq
0  73.199999  73.199999  71.200001  71.399999


In [27]:
results

binary      count      tfidf       freq
0  73.199999  73.199999  71.200001  71.399999

# Training and Testing the Model 2

## MLP Model 2

Now, we will build Multilayer Perceptron (MLP) models to classify encoded documents as either positive or negative.

As you might have expected, the models are simply feedforward network with fully connected layers called `Dense` in the `Keras` library.

Now, we will define our MLP neural network with very little trial and error so cannot be considered tuned for this problem. The configuration is as follows:
- First hidden layer with 100 neurons and Relu activation function
- Dropout layer with p = 0.5
- Output layer with Sigmoid activation function
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

In [21]:
def train_mlp_2(train_x, train_y, batch_size = 50, epochs = 10, verbose =2):
    
    n_words = train_x.shape[1]
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=100, activation='relu', input_shape=(n_words,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=6, activation='softmax')
    ])
    
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(train_x, train_y, batch_size, epochs, verbose)
    return model

## Comparing the Word Scoring Methods

In [24]:
#########################
# Define the vocabulary #
#########################

from collections import Counter
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stemmer = PorterStemmer()
    
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # filter out stop words
    tokens = [w for w in tokens if not w in stopwords]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) >= 1]
    # Stem the token
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens

def add_doc_to_vocab(docs, vocab):
    '''
    input:
        docs: a list of sentences (docs)
        vocab: a vocabulary dictionary
    output:
        return an updated vocabulary
    '''
    for doc in docs:
        tokens = clean_doc(doc)
        vocab.update(tokens)
    return vocab
        
def doc_to_line(doc, vocab):
    tokens = clean_doc(doc)
    # filter by vocab
    tokens = [token for token in tokens if token in vocab]
    line = ' '.join(tokens)
    return line

def clean_docs(docs, vocab):
    lines = []
    for doc in docs:
        line = doc_to_line(doc, vocab)
        lines.append(line)
    return lines

# prepare bag-of-words encoding of docs
def prepare_data(train_docs, test_docs, mode):
    # create the tokenizer
    tokenizer = Tokenizer()
    # fit the tokenizer on the documents
    tokenizer.fit_on_texts(train_docs)
    # encode training data set
    Xtrain = tokenizer.texts_to_matrix(train_docs, mode=mode)
    # encode test data set
    Xtest = tokenizer.texts_to_matrix(test_docs, mode=mode)
    return Xtrain, Xtest

def train_mlp_2(train_x, train_y, batch_size = 50, epochs = 10, verbose =2):
    
    n_words = train_x.shape[1]
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=100, activation='relu', input_shape=(n_words,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=6, activation='softmax')
    ])
    
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(train_x, train_y, batch_size, epochs, verbose)
    return model

# Separate the sentences and the labels for training and testing
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
print('train_x size: ', len(train_x))
print('train_y size: ', len(train_y))

test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)
print('test_x size: ', len(test_x))
print('test_y size: ', len(test_y))

# Run Experiment of 4 different modes
modes = ['binary', 'count', 'tfidf', 'freq']
results = pd.DataFrame()

for mode in modes:
    print('mode: ', mode)
    
    # Instantiate a vocab object
    vocab = Counter()

    # Define a vocabulary for each fold
    vocab = add_doc_to_vocab(train_x, vocab)

    # Clean the sentences
    train_x = clean_docs(train_x, vocab)
    test_x = clean_docs(test_x, vocab)

    # encode data using freq mode
    Xtrain, Xtest = prepare_data(train_x, test_x, mode)

    # train the model
    model = train_mlp_2(Xtrain, train_y, verbose=0, epochs = 8)

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))
    results[mode] = [acc*100]
    
print()
print(results)

train_x size:  5452
train_y size:  5452
test_x size:  500
test_y size:  500
mode:  binary
Test Accuracy: 74.19999837875366
mode:  count
Test Accuracy: 74.59999918937683
mode:  tfidf
Test Accuracy: 71.79999947547913
mode:  freq
Test Accuracy: 73.00000190734863

      binary      count      tfidf       freq
0  74.199998  74.599999  71.799999  73.000002


In [25]:
results

binary      count      tfidf       freq
0  74.199998  74.599999  71.799999  73.000002

# Training and Testing the Model 3

## MLP Model 3

Now, we will build Multilayer Perceptron (MLP) models to classify encoded documents as either positive or negative.

As you might have expected, the models are simply feedforward network with fully connected layers called `Dense` in the `Keras` library.

Now, we will define our MLP neural network with very little trial and error so cannot be considered tuned for this problem. The configuration is as follows:
- First hidden layer with 50 neurons and Relu activation function
- Dropout layer with p = 0.5
- Output layer with Sigmoid activation function
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

In [28]:
def train_mlp_3(train_x, train_y, batch_size = 50, epochs = 10, verbose =2):
    
    n_words = train_x.shape[1]
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=50, activation='relu', input_shape=(n_words,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=6, activation='softmax')
    ])
    
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(train_x, train_y, batch_size, epochs, verbose)
    return model

## Comparing the Word Scoring Methods

In [29]:
#########################
# Define the vocabulary #
#########################

from collections import Counter
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stemmer = PorterStemmer()
    
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # filter out stop words
    tokens = [w for w in tokens if not w in stopwords]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) >= 1]
    # Stem the token
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens

def add_doc_to_vocab(docs, vocab):
    '''
    input:
        docs: a list of sentences (docs)
        vocab: a vocabulary dictionary
    output:
        return an updated vocabulary
    '''
    for doc in docs:
        tokens = clean_doc(doc)
        vocab.update(tokens)
    return vocab
        
def doc_to_line(doc, vocab):
    tokens = clean_doc(doc)
    # filter by vocab
    tokens = [token for token in tokens if token in vocab]
    line = ' '.join(tokens)
    return line

def clean_docs(docs, vocab):
    lines = []
    for doc in docs:
        line = doc_to_line(doc, vocab)
        lines.append(line)
    return lines

# prepare bag-of-words encoding of docs
def prepare_data(train_docs, test_docs, mode):
    # create the tokenizer
    tokenizer = Tokenizer()
    # fit the tokenizer on the documents
    tokenizer.fit_on_texts(train_docs)
    # encode training data set
    Xtrain = tokenizer.texts_to_matrix(train_docs, mode=mode)
    # encode test data set
    Xtest = tokenizer.texts_to_matrix(test_docs, mode=mode)
    return Xtrain, Xtest

def train_mlp_3(train_x, train_y, batch_size = 50, epochs = 10, verbose =2):
    
    n_words = train_x.shape[1]
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=50, activation='relu', input_shape=(n_words,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=6, activation='softmax')
    ])
    
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(train_x, train_y, batch_size, epochs, verbose)
    return model

# Separate the sentences and the labels for training and testing
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
print('train_x size: ', len(train_x))
print('train_y size: ', len(train_y))

test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)
print('test_x size: ', len(test_x))
print('test_y size: ', len(test_y))

# Run Experiment of 4 different modes
modes = ['binary', 'count', 'tfidf', 'freq']
results = pd.DataFrame()

for mode in modes:
    print('mode: ', mode)
    
    # Instantiate a vocab object
    vocab = Counter()

    # Define a vocabulary for each fold
    vocab = add_doc_to_vocab(train_x, vocab)

    # Clean the sentences
    train_x = clean_docs(train_x, vocab)
    test_x = clean_docs(test_x, vocab)

    # encode data using freq mode
    Xtrain, Xtest = prepare_data(train_x, test_x, mode)

    # train the model
    model = train_mlp_3(Xtrain, train_y, verbose=0, epochs = 8)

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))
    results[mode] = [acc*100]
    
print()
print(results)

train_x size:  5452
train_y size:  5452
test_x size:  500
test_y size:  500
mode:  binary
Test Accuracy: 74.40000176429749
mode:  count
Test Accuracy: 75.19999742507935
mode:  tfidf
Test Accuracy: 71.39999866485596
mode:  freq
Test Accuracy: 73.4000027179718

      binary      count      tfidf       freq
0  74.400002  75.199997  71.399999  73.400003


In [30]:
results

binary      count      tfidf       freq
0  74.400002  75.199997  71.399999  73.400003